In [7]:
import os

os.environ['OPENAI_API_KEY']='...'

In [18]:
!pip install openai langchain tiktoken pdfplumber llama_index langchainhub pypdf arxiv faiss-gpu duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 763.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/17/76/47d0cc8161f4bf988583a2839bb1e56baf09d6b80cfa472b9eba4d5f543b/faiss-gpu-1.7.1.post2.tar.gz (from https://pypi.org/simple/faiss-gpu/): Requested faiss-cpu from https://files.pythonhosted.org/packages/17/76/47d0cc8161f4bf988583a2839bb1e56baf09d6b80cfa472b9eba4d5f543b/faiss-gpu-1.7.1.post2.tar.gz has inconsistent name: expected 'faiss-gpu', but metadata has 'faiss-cpu'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 1.3 MB/s eta 0:00:00-:--:--
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/39/8d/b62bc92c8dd4b2a99d4a06b8804280f6445748b6d698eabb037e111080c7/faiss-gpu-1.7.1.post1.tar.gz (from https://pypi.org/simple/faiss-gpu/): Requested faiss-cpu from https://files.pythonhosted.org/packages/39/8d/b62bc92c8dd4b2a99d4a06b8804280f6445748b6d698eabb037e111

# Plan And Execute Agent using Langchain

let's download a paper about agent to have a serious discussion ;)

In [2]:
!pip install arxiv

In [8]:
import arxiv

paper = next(arxiv.Search(id_list=['2303.11366']).results())
paper.download_pdf(filename='agent-reflexion.pdf')

INFO:arxiv.arxiv:Requesting 100 results at offset 0
Requesting 100 results at offset 0
INFO:arxiv.arxiv:Requesting page of results
Requesting page of results
INFO:arxiv.arxiv:Got first page; 1 of inf results available
Got first page; 1 of inf results available


'./agent-reflexion.pdf'

In [9]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Setting up index on the pdf

In [12]:
from langchain.agents import Tool
from langchain.chat_models import ChatOpenAI
from llama_index import VectorStoreIndex, SimpleDirectoryReader

#documents = SimpleDirectoryReader("/content").load_data()  #on colab
documents = SimpleDirectoryReader("../../datatest2").load_data()  #on local

LLAMINDEX Input file: ../../datatest2/agent-reflexion.pdf
PDF reader opening file  ../../datatest2/agent-reflexion.pdf


In [13]:
prompt = """
You are an expert scientist studying language models and can explain complex topics by breaking it down into simpler elements. Your role is to help humans understand scientific papers about language model, in a clear way. Do your best to explain the question asked.
"""

In [14]:

from llama_index import ServiceContext,  OpenAIEmbedding
from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser

llm = ChatOpenAI(model='gpt-4', temperature=0)
embed_model = OpenAIEmbedding()
node_parser = SimpleNodeParser.from_defaults(
  text_splitter=TokenTextSplitter(chunk_size=2048, chunk_overlap=128))

service_context = ServiceContext.from_defaults(
  llm=llm,
  embed_model=embed_model,
  node_parser=node_parser,
  system_prompt=prompt
)

index = VectorStoreIndex.from_documents(documents=documents, show_progress=False,service_context=service_context)

In [6]:
!pip install langchain-experimental numexpr

  Obtaining dependency information for langchain-experimental from https://files.pythonhosted.org/packages/4d/6a/20f8ee4849a3d1cbc707a2f0cc504711932c7b4689498a7f9bf37cb0de95/langchain_experimental-0.0.27-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 759.4 kB/s eta 0:00:00a 0:00:01


In [20]:
!pip install duckduckgo-search


  Obtaining dependency information for duckduckgo-search from https://files.pythonhosted.org/packages/44/63/410089984cf85455e92eebd56c8a22c33c7d6a89a51ea99aae5359aa31f4/duckduckgo_search-3.9.2-py3-none-any.whl.metadata
  Obtaining dependency information for aiofiles>=23.2.1 from https://files.pythonhosted.org/packages/c5/19/5af6804c4cc0fed83f47bff6e413a98a36618e7d40185cd36e69737f3b0e/aiofiles-23.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for lxml>=4.9.3 from https://files.pythonhosted.org/packages/ed/62/ffc30348ae141f69f9f23b65ba769db7ca209856c9a9b3406279e0ea24de/lxml-4.9.3-cp311-cp311-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for httpx[brotli,http2,socks]>=0.25.0 from https://files.pythonhosted.org/packages/33/0d/d9ce469af019741c8999711d36b270ff992ceb1a0293f73f9f34fdf131e9/httpx-0.25.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore<0.19.0,>=0.18.0 from https://files.pythonhosted.org/packages/ac/97/724afbb7925

### Defining a set of tools

1. semantic search on pdf
2. internet search
3. calculator

In [21]:

from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import Tool
from langchain.chains import LLMMathChain

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about Reflexion: Language Agents with Verbal Reinforcement Learning. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
    Tool(
        name="DuckDuckGoSearch",
        func=DuckDuckGoSearchRun().run,
        description="Search the web with DuckDuckGo"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

### Planner, Executor and Action Agent

https://python.langchain.com/docs/use_cases/more/agents/autonomous_agents/plan_and_execute



In [24]:

#----------------------
# MEMORY and LLM
#----------------------

from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.chat_models import ChatOpenAI
# This is needed for both the memory and the prompt
memory_key = "history"
memory = AgentTokenBufferMemory(memory_key=memory_key, llm=llm)
llm = ChatOpenAI(model='gpt-4', temperature = 0)
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)



In [89]:
#-----------------
# PROMPT TEMPLATE
#-----------------

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
# from langchain.agents import ConversationalChatAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

prompt_2 = """
Do your best to answer the questions.
Feel free to use any tools available to look up relevant information, only if neccessary.
"""

system_message = SystemMessage(
        content=(
          prompt_2
        )
)

prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
    )

#-----------------
# AGENT
#-----------------

agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

In [90]:
#-----------------
# AGENT EXECUTOR
#-----------------

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True,
                                   return_intermediate_steps=True)

In [91]:
agent_executor({"input": "hi, i am Isabelle"})



> Entering new AgentExecutor chain...
Hello Isabelle! How can I assist you today?

> Finished chain.


{'input': 'hi, i am Isabelle',
 'history': [HumanMessage(content='hi, i am bob'),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='hi, i am Isabelle'),
  AIMessage(content='Hello Isabelle! How can I assist you today?')],
 'output': 'Hello Isabelle! How can I assist you today?',
 'intermediate_steps': []}

In [92]:
agent_executor({"input":"I want to build a reflexion agent, how shall I proceed?"})



> Entering new AgentExecutor chain...

Invoking: `LlamaIndex` with `How to build a reflexion agent?`


To build a Reflexion agent, you would need to follow a framework that uses verbal reinforcement to help the agent learn from previous mistakes. This process involves converting binary or scalar feedback from the environment into natural language, which the agent then uses to reflect on its actions. The agent maintains this reflective text in an episodic memory buffer, which it uses to make better decisions in subsequent trials. This approach allows the agent to learn quickly and efficiently from trial-and-error without the need for extensive training samples or expensive model fine-tuning. The Reflexion framework is flexible enough to incorporate various types and sources of feedback signals, including scalar values or free-form language, and external or internally simulated signals.


> Finished chain.


{'input': 'I want to build a reflexion agent, how shall I proceed?',
 'history': [HumanMessage(content='hi, i am bob'),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='hi, i am Isabelle'),
  AIMessage(content='Hello Isabelle! How can I assist you today?'),
  HumanMessage(content='I want to build a reflexion agent, how shall I proceed?'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'LlamaIndex', 'arguments': '{\n  "__arg1": "How to build a reflexion agent?"\n}'}}),
  FunctionMessage(content='To build a Reflexion agent, you would need to follow a framework that uses verbal reinforcement to help the agent learn from previous mistakes. This process involves converting binary or scalar feedback from the environment into natural language, which the agent then uses to reflect on its actions. The agent maintains this reflective text in an episodic memory buffer, which it uses to make better decisions in subsequent trials. This app

# Knowledge graph query engine

https://siwei.io/graph-enabled-llama-index/knowledge_graph_query_engine.html



In [93]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [94]:
from llama_index import (
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
    SimpleDirectoryReader,
)
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

In [95]:
from langchain import OpenAI
from IPython.display import Markdown, display

## Setting Up the llama_index service context

In [98]:
from llama_index import (
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
    SimpleDirectoryReader,
)
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

from langchain.chat_models import ChatOpenAI
from IPython.display import Markdown, display

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size=512)

## Prepare for NebulaGraph  

Before next step to creating the Knowledge Graph, let's ensure we have a running NebulaGraph with defined data schema.

In [ ]:
%pip install ipython-ngql nebula3-python


  Obtaining dependency information for ipython-ngql from https://files.pythonhosted.org/packages/4a/1a/11cfe058ae84df31f503d115941fe665b116ef6fee3488ba284d03ba45c3/ipython_ngql-0.7.5-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.4/312.4 kB 6.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 8.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492023 sha256=91bc2f20856912961218ecbc92681ff9a01bcaede4a505a191954f74382d0934
  Stored in directory: /home/vscode/.cache/pip/wheels/da/19/ca/9d8c44cd311a955509d7e13da3f0bea42400c469ef825b580b
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Create a NebulaGraph cluster with:
    # Option 0 for machines with Docker: `curl -fsSL nebula-up.siwei.io/install.sh | bash`
    # Option 1 for Desktop: NebulaGraph Docker Extension https://hub.docker.com/extensions/weygu/nebulagraph-dd-ext

# Once the nebula Graph cluster is installed, execute the following in the nebula graph console:
#(1)
    # CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
#(2) Select create space and run the following to define your schema
    # USE llamaindex;
    # CREATE TAG entity(name string);
    # CREATE EDGE relationship(relationship string);
    # CREATE TAG INDEX entity_index ON entity(name(256));

## ---> please input the correct values for nebula graphs, these are the default values for localhost

import os
os.environ['NEBULA_USER'] = "root"
os.environ['NEBULA_PASSWORD'] = "nebula" # default is "nebula
os.environ['NEBULA_ADDRESS'] = "192.168.1.198:9669" # assumed we have NebulaGraph installed locally

space_name = "llamaindex"
edge_types, rel_prop_names = ["relationship"], ["relationship"] # default, could be omit if create from an empty kg
tags = ["entity"] # default, could be omit if create from an empty kg

Prepare for StorageContext with graph_store as NebulaGraphStore

In [ ]:
graph_store = NebulaGraphStore(space_name=space_name, edge_types=edge_types, rel_prop_names=rel_prop_names, tags=tags)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

## Build the Knowledge Graph with LlamaIndex  
With the help of Llama Index and LLM defined, we could build Knowledge Graph from given documents.

If we have a Knowledge Graph on NebulaGraphStore already, this step could be skipped

### Step 1, load data from Wikipedia for "Guardians of the Galaxy Vol. 3"

In [ ]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(pages=['Guardians of the Galaxy Vol. 3'], auto_suggest=False)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): en.wikipedia.org:80
Starting new HTTP connection (1): en.wikipedia.org:80
Starting new HTTP connection (1): en.wikipedia.org:80
DEBUG:urllib3.connectionpool:http://en.wikipedia.org:80 "GET /w/api.php?prop=info%7Cpageprops&inprop=url&ppprop=disambiguation&redirects=&titles=Guardians+of+the+Galaxy+Vol.+3&format=json&action=query HTTP/1.1" 301 0
http://en.wikipedia.org:80 "GET /w/api.php?prop=info%7Cpageprops&inprop=url&ppprop=disambiguation&redirects=&titles=Guardians+of+the+Galaxy+Vol.+3&format=json&action=query HTTP/1.1" 301 0
http://en.wikipedia.org:80 "GET /w/api.php?prop=info%7Cpageprops&inprop=url&ppprop=disambiguation&redirects=&titles=Guardians+of+the+Galaxy+Vol.+3&format=json&action=query HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): en.wikipedia.org:443
Starting new HTTPS connection (1): en.wikipedia.org:443
Starting new HTTPS connection (1): en.wikipedia.org:443
DEBUG:urllib3.conne

### Step 2, Generate a KnowledgeGraphIndex with NebulaGraph as graph_store

Then, we will create a KnowledgeGraphIndex to enable Graph based RAG.  
Apart from that, we have a Knowledge Graph up and running for other purposes, too!  

See here also: https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html



In [ ]:
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    service_context=service_context,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Guardians of the Galaxy Vol. 3 (stylized in mar...
> Adding chunk: Guardians of the Galaxy Vol. 3 (stylized in mar...
> Adding chunk: Guardians of the Galaxy Vol. 3 (stylized in mar...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: After Adam critically wounds Rocket, he is stab...
> Adding chunk: After Adam critically wounds Rocket, he is stab...
> Adding chunk: After Adam critically wounds Rocket, he is stab...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: He destroys Counter-Earth, killing the Humanima...
> Adding chunk: He destroys Counter-Earth, killing the Humanima...
> Adding chunk: He destroys Counter-Earth, killing the Humanima...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: In a mid-credits scene, the new Guardians, cons...
> Adding chunk: In a mid-credits scene, the new Guardians, cons...
> Adding chunk: In a mid-credits scene, the new Guardians, cons...
DEBUG:llama_index.node_parser.no

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=813 request_id=c86d44311bc71424317155fc9c3d22da response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=813 request_id=c86d44311bc71424317155fc9c3d22da response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=813 request_id=c86d44311bc71424317155fc9c3d22da response_code=200
DEBUG:llama_index.llm_predictor.base:(Guardians of the Galaxy, are attacked by, Adam)
(Adam, seeks to destroy, Guardians of the Galaxy)
(Adam, is created by, Ayesha)
(Rocket, is critically wounded by, Adam)
(Guardians of the Galaxy, are attacked by, Adam)
(Adam, seeks to destroy, Guardians of t

Now we have a Knowledge Graph on NebulaGraph cluster under space named llamaindex about the 'Guardians of the Galaxy Vol. 3' movie, let's play with it a little bit.

https://siwei.io/en/nebulagraph-in-jupyter-notebook/


In [ ]:
# install related packages, password is nebula by default
%pip install ipython-ngql networkx pyvis


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 13.6 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for jsonpickle>=1.4.1 from https://files.pythonhosted.org/packages/d3/25/6e0a450430b7aa194b0f515f64820fc619314faa289458b7dfca4a026114/jsonpickle-3.0.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Connection Pool Created
INFO:nebula3.logger:Get connection to ('192.168.1.198', 9669)
Get connection to ('192.168.1.198', 9669)


,Name
0,llamaindex


#### Create graph from index  

In [ ]:
## create graph
from pyvis.network import Network

g = kg_index.get_networkx_graph(200)
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("example.html")

#### using ngql

In [ ]:
%load_ext ngql
%ngql --address 192.168.1.198 --port 9669 --user root --password nebula

The ngql extension is already loaded. To reload it, use:
  %reload_ext ngql
Connection Pool Created
INFO:nebula3.logger:Get connection to ('192.168.1.198', 9669)
Get connection to ('192.168.1.198', 9669)


,Name
0,llamaindex


The result of ngql commands is always _  , by default the result is returned as a pandas DataFrame.  
This said, this is only valid just after you execute the cell, later than that it _ becomes the result of another cell.

In [ ]:
type(_)

pandas.core.frame.DataFrame

To change the result type and get the raw output use the following:

In [ ]:
# %config IPythonNGQL.ngql_result_style="raw"

In [ ]:
# Query some random Relationships with Cypher
%ngql USE llamaindex;
%ngql MATCH ()-[e]->() RETURN e LIMIT 10

INFO:nebula3.logger:Get connection to ('192.168.1.198', 9669)
Get connection to ('192.168.1.198', 9669)
INFO:nebula3.logger:Get connection to ('192.168.1.198', 9669)
Get connection to ('192.168.1.198', 9669)


,e
0,"(""Adam McKay"")-[:relationship@6555594894570341..."
1,"(""Callie Brand"")-[:relationship@-8033826305454..."
2,"(""Deadline Hollywood"")-[:relationship@-5834913..."
3,"(""Disney"")-[:relationship@353711719953188304{r..."
4,"(""Disney+ Star Wars series The Mandalorian"")-[..."
5,"(""Disney+ Star Wars series The Mandalorian"")-[..."
6,"(""Elizabeth Debicki"")-[:relationship@179912200..."
7,"(""Feige and Pratt"")-[:relationship@35938744641..."
8,"(""Gillan"")-[:relationship@7032484236953577661{..."
9,"(""Groot"")-[:relationship@-8550268596295077678{..."


In [ ]:
pyvis_graph=_

In [ ]:
from pyvis import network
import networkx as nx

g=network.Network(notebook=True, cdn_resources='in_line')
nxg = nx.complete_graph(5)
g.from_nx(nxg)
# g.show('example.html')

## Asking the Knowledge Graph   
Finally, let's demo how to Query Knowledge Graph with Natural language!   

Here, we will leverage the KnowledgeGraphQueryEngine, with NebulaGraphStore as the storage_context.graph_store   

In [ ]:
from llama_index.query_engine import KnowledgeGraphQueryEngine

# from llama_index.storage.storage_context import StorageContext
# from llama_index.graph_stores import NebulaGraphStore

query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

DEBUG:llama_index.graph_stores.nebulagraph:get_schema()
schema: Node properties: [{'tag': 'entity', 'properties': [('name', 'string')]}]
Edge properties: [{'edge': 'relationship', 'properties': [('relationship', 'string')]}]
Relationships: ['(:entity)-[:relationship]->(:entity)']

get_schema()
schema: Node properties: [{'tag': 'entity', 'properties': [('name', 'string')]}]
Edge properties: [{'edge': 'relationship', 'properties': [('relationship', 'string')]}]
Relationships: ['(:entity)-[:relationship]->(:entity)']

get_schema()
schema: Node properties: [{'tag': 'entity', 'properties': [('name', 'string')]}]
Edge properties: [{'edge': 'relationship', 'properties': [('relationship', 'string')]}]
Relationships: ['(:entity)-[:relationship]->(:entity)']



In [ ]:
response = query_engine.query(
    "Tell me about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["\\nGenerate NebulaGraph query from natural language.\\nUse only the provided relationship types and properties in the schema.\\nDo not use any other relationship types or properties that are not provided.\\nSchema:\\n---\\nNode properties: [{\'tag\': \'entity\', \'properties\': [(\'name\', \'string\')]}]\\nEdge properties: [{\'edge\': \'relationship\', \'properties\': [(\'relationship\', \'string\')]}]\\nRelationships: [\'(:entity)-[:relationship]->(:entity)\']\\n\\n---\\nNote: NebulaGraph speaks a dialect of Cypher, comparing to standard Cypher:\\n\\n1. it uses double equals sign for comparison: `==` rather than `=`\\n2. it needs explicit label specification when refe

<b>

Peter Quill is a character in the Marvel Cinematic Universe.</b>

### Translate natural language into queries

In [ ]:
graph_query = query_engine.generate_query(
    "Tell me about Peter Quill?",
)

display(Markdown(f"""
```cypher
{graph_query}
```
"""))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["\\nGenerate NebulaGraph query from natural language.\\nUse only the provided relationship types and properties in the schema.\\nDo not use any other relationship types or properties that are not provided.\\nSchema:\\n---\\nNode properties: [{\'tag\': \'entity\', \'properties\': [(\'name\', \'string\')]}]\\nEdge properties: [{\'edge\': \'relationship\', \'properties\': [(\'relationship\', \'string\')]}]\\nRelationships: [\'(:entity)-[:relationship]->(:entity)\']\\n\\n---\\nNote: NebulaGraph speaks a dialect of Cypher, comparing to standard Cypher:\\n\\n1. it uses double equals sign for comparison: `==` rather than `=`\\n2. it needs explicit label specification when refe


```cypher
```
MATCH (p:`entity`)-[:relationship]->(m:`entity`) WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p.`entity`.`name`;
```
```


In [ ]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p.`entity`.`name`, e.relationship, m.`entity`.`name`;

INFO:nebula3.logger:Get connection to ('192.168.1.198', 9669)
Get connection to ('192.168.1.198', 9669)
Get connection to ('192.168.1.198', 9669)


,p.entity.name,e.relationship,m.entity.name
0,Peter Quill,would return to the MCU,May 2021


# Knowledge graph and semantic search index

https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html



In [ ]:
import logging
import sys
logging.basicConfig(
    stream=sys.stdout, level=logging.DEBUG
)  # logging.DEBUG for more verbose output

from llama_index import (
    KnowledgeGraphIndex,
    ServiceContext,
    SimpleDirectoryReader,
    VectorStoreIndex
)
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore


from llama_index.llms import OpenAI
from IPython.display import Markdown, display

## Prepare Nebula graph

In [ ]:
import os
os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"
os.environ[
    "NEBULA_ADDRESS"
] = "172.16.0.245:9669"  # assumed we have NebulaGraph 3.5.0 or newer installed locally

# Assume that the graph has already been created
# Create a NebulaGraph cluster with:
# Option 0: `curl -fsSL nebula-up.siwei.io/install.sh | bash`
# Option 1: NebulaGraph Docker Extension https://hub.docker.com/extensions/weygu/nebulagraph-dd-ext
# and that the graph space is called "llamaindex"
# If not, create it with the following commands from NebulaGraph's console:
# CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
# :sleep 10;
# USE llamaindex;
# CREATE TAG entity(name string);
# CREATE EDGE relationship(relationship string);
# CREATE TAG INDEX entity_index ON entity(name(256));

space_name = "llamaindex_2"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

## Load data from Wikipedia

In [ ]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents_science= loader.load_data(pages=["2023 in science"], auto_suggest=False)

In [ ]:
len(documents_science[0].text)

54082

## Service context for loading documents and creating knowledge graph

In [ ]:

from llama_index.embeddings import OpenAIEmbedding
from llama_index.node_parser import SimpleNodeParser
from llama_index import ServiceContext, LLMPredictor, OpenAIEmbedding, PromptHelper
from llama_index.llms import OpenAI
from llama_index.text_splitter import TokenTextSplitter

# define LLM
# NOTE: at the time of demo, text-davinci-002 did not have rate-limit errors
llm = OpenAI(temperature=0, model="text-davinci-002")

embed_model = OpenAIEmbedding()
node_parser = SimpleNodeParser.from_defaults(
  text_splitter=TokenTextSplitter(chunk_size=512, chunk_overlap=64))

service_context = ServiceContext.from_defaults(
  llm=llm,
  embed_model=embed_model,
  node_parser=node_parser
)

## Create knowledge graph index

In [ ]:
graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)

(National Disability Insurance Scheme, Pricing Arrangements and Price Limits, 2023-24)
(Pricing Arrangements, valid from, 1 July 2023)
(Version, 1.0, Released 16 June 2023)
(NDIS Pricing Arrangements and Price Limits 2023-24, is, document)
(NDIS Pricing Arrangements and Price Limits 2023-24, published, 16/06/2023)
(NDIS Pricing Arrangements and Price Limits 2023-24, has date of effect, 1 July 2023)
(NDIS Pricing Arrangements and Price Limits, are, price regulation documents)
(NDIS Pricing Arrangements and Price Limits, contain, support items)
(NDIS Pricing Arrangements and Price Limits, contain, support purposes)
(NDIS Pricing Arrangements and Price Limits, contain, support categories)
(NDIS Pricing Arrangements and Price Limits, contain, registration groups)
(NDIS Pricing Arrangements and Price Limits, contain, units of measure)
(NDIS Pricing Arrangements and Price Limits, contain, general claiming rules)
(NDIS Pricing Arrangements and Price Limits, contain, service agreements)
(NDIS 

In [ ]:
G = kg_index.get_networkx_graph(limit = 10000)

In [ ]:
from helpers import pickleSave
pickleSave(G, 'NDISPriceGuideKG_nx', folder='.', silent=False)

Saving object NDISPriceGuideKG_nx to pickle file ./NDISPriceGuideKG_nx.pkl


## Checking the KG

In [ ]:
# Service context for querying the KG


from llama_index.embeddings import OpenAIEmbedding
from llama_index.node_parser import SimpleNodeParser
from llama_index import ServiceContext, LLMPredictor, OpenAIEmbedding, PromptHelper
from llama_index.llms import OpenAI
from llama_index.text_splitter import TokenTextSplitter

# define LLM
# NOTE: at the time of demo, text-davinci-002 did not have rate-limit errors
llm = OpenAI(temperature=0, model="gpt-4")

embed_model = OpenAIEmbedding()
# node_parser = SimpleNodeParser.from_defaults(
#   text_splitter=TokenTextSplitter(chunk_size=512, chunk_overlap=64))

service_context_querying_kg = ServiceContext.from_defaults(
  llm=llm,
  embed_model=embed_model,
#   node_parser=node_parser
)

In [ ]:
storage_context.graph_store.persist('NdisPriceGuide_512_64_graph.txt')

In [ ]:
from llama_index.query_engine import KnowledgeGraphQueryEngine

# from llama_index.storage.storage_context import StorageContext
# from llama_index.graph_stores import NebulaGraphStore

query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    service_context=service_context_querying_kg,
    llm=llm,
    verbose=True,
)

In [ ]:
graph_query = query_engine.generate_query(
    "What is the support item code for invoicing gardening for a participant?",
)

display(Markdown(f"""
```cypher
{graph_query}
```
"""))


```cypher
The provided schema does not contain any information related to "support item code", "invoicing", "gardening", or "participant". Therefore, it's not possible to generate a NebulaGraph query based on the provided schema and the given question.
```


## Create vector store index

In [ ]:
documents[1].text

' \nNDIS Pricing Arrangements and Price Limits 202 3-24 Version 1.0  (published 16/06/2023 )  Page 2 of 101 Copyright  \n© National Disability Insurance Agency 202 3 \nUse of National Disability Insurance Agency copyright material  \nThe material in this document  with the exception of logos, trademarks, third party material and \nother content as specified is licensed under Creative Commons CC NC licence, version 4.0. With \nthe exception of logos, trademarks, third party material and other content as specified, you may \nreproduce the material in this document , provided you a cknowledge the National Disability \nInsurance Agency as the owner of all intellectual property rights in the reproduced material by \nusing ‘© National Disability Insurance Agency 2023’ and do not use the material for commercial \npurposes.  \nReproduction of any Cr eative Commons material in this document is subject to the CC NC licence \nconditions available on the Creative Commons site, as is the full legal

In [ ]:
vector_index = VectorStoreIndex.from_documents(documents)

## Define a CustomRetriever  


The purpose of this demo was to test the effectiveness of using Knowledge Graph queries for retrieving information that is distributed across multiple nodes in small pieces. To achieve this, we adopted a simple approach: performing retrieval on both sources and then combining them into a single context to be sent to LLM.

Thanks to the flexible abstraction provided by Llama Index Retriever, implementing this approach was relatively straightforward. We created a new class called CustomRetriever which retrieves data from both VectorIndexRetriever and KGTableRetriever

In [ ]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever

from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [ ]:

from llama_index.query_engine import RetrieverQueryEngine

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)



## Create Query Engines  

To enable comparsion, we also create vector_query_engine, kg_keyword_query_engine together with our custom_query_engine.

In [ ]:
llm = OpenAI(temperature=0, model="gpt-4")
embed_model = OpenAIEmbedding()

service_context_querying_engine = ServiceContext.from_defaults(  # useful?
  llm=llm,
  embed_model=embed_model,
  system_prompt=prompt_llamaindex_retriever)


In [ ]:
from llama_index import get_response_synthesizer
# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context_querying_engine,
    response_mode="tree_summarize",
)
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

vector_query_engine = vector_index.as_query_engine(service_context=service_context_querying_engine)

kg_keyword_query_engine = kg_index.as_query_engine(
    # setting to false uses the raw triplets instead of adding the text from the corresponding nodes
    include_text=False,
    retriever_mode="keyword",
    response_mode="tree_summarize",
    service_context=service_context_querying_engine # useful?
)

## Query with different retrievers

With the above query engines created for corresponding retrievers, let’s see how they perform.

First, we go with the pure knowledge graph.

In [ ]:
query = "I have mowed the lawn of 1 participant on Saturday 9 to 11 am last week, how should I invoice NDIS?"

In [ ]:
response = kg_keyword_query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

<b>To invoice NDIS for the lawn mowing service you provided, you need to follow the NDIS Pricing Arrangements and Price Limits. However, I need more information to provide the correct support item code and maximum price. Specifically, I need to know the location where the service was provided and whether it falls under a specific support category. 

Once I have this information, I can provide the appropriate support item code, the maximum price you can charge for the service, and any specific rules related to claiming for this service.</b>

Then the vector store approach.

In [ ]:
response = vector_query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

<b>To invoice NDIS for the lawn mowing service you provided to a participant on a Saturday, you need to follow these steps:

### Support Item Code
First, you need to identify the appropriate support item code for the lawn mowing service. This code will depend on the specific category of support that lawn mowing falls under in the NDIS Price Guide. If you're unsure, you may need to consult the NDIS Price Guide or ask for assistance from an NDIS representative.

### Maximum Price
The maximum price you can charge will depend on the support item code and the location where the service was provided. The NDIS Price Guide provides a list of maximum prices for each support item, which varies by location. 

### Specific Rules
As the service was provided on a Saturday, it falls under the "Saturday Support" category as per the NDIS Pricing Arrangements and Price Limits. Therefore, you should invoice it as a "Saturday Support". 

Remember, if the service crosses a shift boundary and the same worker delivers the entire support, the higher of the relevant price limits applies to the entire support. You should discuss this billing arrangement with the participant.

Finally, ensure that the proposed charges for the activities comply with the NDIS Pricing Arrangements and Price Limits, and that you have the agreement of the participant in advance.</b>

In [ ]:
response = custom_query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

<b>To invoice NDIS for the lawn mowing service you provided to a participant on a Saturday, you need to follow these steps:

### Support Item Code
First, you need to identify the appropriate support item code for the lawn mowing service. This code should be listed in the NDIS Price Guide under the category of 'Yard Maintenance' or similar. If you're unsure of the exact code, please provide more details about the service you provided.

### Maximum Price
The maximum price you can charge will depend on the specific support item code and the location where the service was provided. The NDIS Pricing Arrangements and Price Limits document outlines the maximum prices for different services and locations.

### Specific Rules
When claiming for this service, you need to ensure that the proposed charges comply with the NDIS Pricing Arrangements and Price Limits. Also, the service agreement with the participant should specify that such services can be claimed. 

Remember, the claim should be made using the same support item as would have been used if the support had been delivered. 

Please note that the information provided here is general in nature. For specific advice related to your situation, you may need to consult with an NDIS representative or a professional advisor.</b>

In [ ]:
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="tree_summarize",
)
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

In [ ]:
response = custom_query_engine.query("What are plan managers?")
display(Markdown(f"<b>{response}</b>"))

<b>Response 1: 

Plan managers are individuals who help participants with the management of their plans. This includes tasks such as budgeting, claims management, and provider payments.
---------------------
Response 2: 

Plan managers are responsible for ensuring that providers do not charge participants more than the price limits set by the NDIA. They must also provide the NDIA with the Australian Business Number (ABN) of the service provider delivering the support.
---------------------
Response 3: 

Plan managers are responsible for managing NDIS plans and ensuring that participants receive the support they need. They are not responsible for ensuring that providers are TTP compliant.</b>